In [17]:
# importing packages
import pandas as pd
import numpy as np

In [18]:
import os
root = os.getcwd()

In [19]:
# load photo and business data
photo =  pd.read_json(root + '/../yelp/photos.json', lines= True)
df = pd.read_json(root + '/../yelp/yelp_academic_dataset_business.json', lines=True)

In [20]:
df.dropna(inplace = True)

In [21]:
# inspect photo dataset
photo.head()

,photo_id,business_id,caption,label
0,zsvj7vloL4L5jhYyPIuVwg,Nk-SJhPlDBkAZvfsADtccA,Nice rock artwork everywhere and craploads of ...,inside
1,HCUdRJHHm_e0OCTlZetGLg,yVZtL5MmrpiivyCIrVkGgA,,outside
2,vkr8T0scuJmGVvN2HJelEA,_ab50qdWOk0DdB6XOrBitw,oyster shooter,drink
3,pve7D6NUrafHW3EAORubyw,SZU9c8V2GuREDN5KgyHFJw,Shrimp scampi,food
4,H52Er-uBg6rNrHcReWTD2w,Gzur0f0XMkrVxIwYJvOt2g,,food


df1 = photo[photo.label == 'drink'][['photo_id', 'label']].rename({'label':'classes'}, axis = 1)

df1.classes = 0

In [22]:
# see what class in photo label column
photo.label.unique()

array(['inside', 'outside', 'drink', 'food', 'menu'], dtype=object)

In [23]:
# use label column containing only the food
photo = photo.loc[photo.label == 'food']

In [24]:
# turn photo caption column with no caption to Null
photo.caption = photo.caption.apply(lambda x: 'NONE' if x == '' else x)

In [25]:
# drop all row without caption
photo = photo[photo.caption != 'NONE']

# drop label column
photo.drop('label', axis = 1, inplace=True)

# merge photo dataset with business dataset
merge = pd.merge(df,photo, on='business_id')

In [26]:
df = df[df.categories.str.contains('restaurant', case=False)]

In [27]:
# explode the categories into a list of category
df['cat'] = df.categories.explode().apply(lambda x: x.split(', '))

# define a list to store category without duplicate
store = []
for cats in iter(df.cat):
    for cat in cats:
        if cat not in store:
            store.append(cat)
            
print(len(store), store)

701 ['Restaurants', 'Food', 'Bubble Tea', 'Coffee & Tea', 'Bakeries', 'Burgers', 'Fast Food', 'Sandwiches', 'Ice Cream & Frozen Yogurt', 'Vietnamese', 'Food Trucks', 'American (Traditional)', 'Diners', 'Breakfast & Brunch', 'Delis', 'Italian', 'Sushi Bars', 'Japanese', 'Korean', 'Cafes', 'Bars', 'Wine Bars', 'Nightlife', 'Steakhouses', 'Asian Fusion', 'Seafood', 'Cocktail Bars', 'Pizza', 'Chicken Wings', 'Salad', 'Soup', 'Eatertainment', 'Arts & Entertainment', 'Brewpubs', 'Breweries', 'Specialty Food', 'Pasta Shops', 'Chinese', 'Event Planning & Services', 'Caterers', 'Pets', 'Pet Adoption', 'American (New)', 'Sports Bars', 'Beer Bar', 'Lounges', 'Wraps', 'Automotive', 'Gas Stations', 'Convenience Stores', 'Pubs', 'Gastropubs', 'Venues & Event Spaces', 'Juice Bars & Smoothies', 'Fruits & Veggies', 'Sporting Goods', 'Sports Wear', 'Fashion', 'Shopping', 'Cajun/Creole', 'French', 'Moroccan', 'Mediterranean', 'Live/Raw Food', 'Beer', 'Wine & Spirits', 'Filipino', 'Mexican', 'Barbeque', '

# getting the class we wanted for image classification
def find_class(x):
    #if 'chicken' in x:
    #    return 0
    if 'steak' in x or 'chicken' in x or 'meat' in x:
        return 0
    #elif 'sushi' in x:
    #    return 1
    elif 'veg' in x or 'salad' in x:
        return 1
    else:
        return 'None'

df['classes'] = df.categories.apply(lambda x: find_class(x.lower()))
df['classes'].value_counts()

In [28]:
# getting the class we wanted for image classification
def find_class(x):
    if 'steak' in x or 'chicken' in x or 'meat' in x:
        return 0
    elif 'veg' in x or 'salad' in x:
        return 1
    #elif 'pizza' in x:
    #    return 2
    #elif 'burger' in x:
    #    return 3
    else:
        return 'None'

df['classes'] = df.categories.apply(lambda x: find_class(x.lower()))
df['classes'].value_counts()

None    35822
0        5072
1        3798
Name: classes, dtype: int64

In [29]:
# drop none classes
df = df[df.classes != 'None']

In [30]:
photo_processed = pd.merge(photo[['photo_id', 'business_id']], df[['business_id', 'classes']]).drop_duplicates().drop('business_id', axis =1)

In [31]:
photo_processed.classes.value_counts()

1    6166
0    5777
Name: classes, dtype: int64

In [32]:
photo_processed.to_csv('processed_photo.csv', index = False)